## Dependencias 

In [1]:
import numpy as np
import pandas as pd 

from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,CHAR,INTEGER,BIGINT,BOOLEAN,DATE,DATETIME,FLOAT

import json
pd.set_option('display.max_columns',None)

## Lectura de datos 

In [2]:
ruta = '/media/jose/090f6b94-de30-4aaf-9f8a-4e18b120d7f6/bd/02.  Para Ingeniería/cc_txn/credit_card_transactions-ibm_v2.csv'

In [3]:
data = pd.read_csv(ruta,dtype=str)
data.shape

(24386900, 15)

In [4]:
data.head(2)

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No


In [85]:
data.insert(0,'id',data.index+1)

## Conexión a BD

In [4]:
creds = json.load(open('../../creds/mysql.json','r'))

In [5]:
cnx = create_engine(f"mysql+pymysql://{creds['u']}:{creds['p']}@{creds['h']}/{creds['d']}").connect()

## Entidades

### Comercio

In [6]:
entComercio = data[['Merchant Name']].drop_duplicates().reset_index(drop=True)
entComercio.insert(0,'id',entComercio.index+1)
entComercio.columns = ['id','merchant_name']
entComercio['merchant_name'] = entComercio['merchant_name'].map(str)
entComercio.head()

,id,merchant_name
0,1,3527213246127876953
1,2,-727612092139916043
2,3,3414527459579106770
3,4,5817218446178736267
4,5,-7146670748125200898


In [7]:
entComercio['merchant_name'].map(len).max()

20

In [8]:
dtypes = [INTEGER,VARCHAR(20)]
entComercio.to_sql(name='tbl_merchant',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,
               dtype=dict(zip(entComercio.columns,dtypes)))

100343

### Territorio

In [9]:
entTerritorio = pd.read_clipboard()

In [11]:
entTerritorio.tail()

,id,territory_code,territory_name
220,221,170,Paraguay
221,222,171,Kiribati
222,223,172,Botswana
223,224,173,Online
224,225,AA,Diplomatic Post Office


In [12]:
dtypes = [INTEGER,VARCHAR(3),VARCHAR(32)]
entTerritorio.to_sql(name='tbl_territory',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,
               dtype=dict(zip(entTerritorio.columns,dtypes)))

225

### Ciudad

In [13]:
data['Merchant State'] = np.where(data['Merchant City']=='ONLINE','Online',data['Merchant State'])

In [14]:
entCiudad = data[['Merchant City','Merchant State']].drop_duplicates()

In [15]:
entCiudad.columns = ['territory_city','territory_name']

In [16]:
aux = entCiudad.merge(entTerritorio,how='left',on=['territory_name'])
aux['id'] =np.where(aux['id']<52,None,aux['id']) 
aux = aux.merge(entTerritorio,how='left',
                left_on=['territory_name'],right_on=['territory_code'])
aux.tail()


,territory_city,territory_name_x,id_x,territory_code_x,id_y,territory_code_y,territory_name_y
20324,Loysville,PA,NaN,NaN,7.0,PA,Pennsylvania
20325,Laurel Bloomery,TN,NaN,NaN,14.0,TN,Tennessee
20326,Alburgh,VT,NaN,NaN,48.0,VT,Vermont
20327,Buskirk,NY,NaN,NaN,4.0,NY,New York
20328,Mooers,NY,NaN,NaN,4.0,NY,New York


In [17]:
aux['id_territory'] = (np.where(aux['id_y'].isnull(),aux['id_x'],aux['id_y'])).astype(int)

In [18]:
aux = aux[['id_territory','territory_city']].dropna().reset_index(drop=True)
aux.insert(0,'id',aux.index+1)

In [19]:
aux['territory_city'].map(len).max()

26

In [20]:
dtypes = [INTEGER,INTEGER,VARCHAR(26)]
aux.to_sql(name='tbl_city',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,
               dtype=dict(zip(aux.columns,dtypes)))

20328

### Tipo de Transacción


In [21]:
entTipoTxn = data[['Use Chip']].drop_duplicates().reset_index(drop=True)
entTipoTxn.insert(0,'id',entTipoTxn.index+1)
entTipoTxn.columns = ['id','txn_type']
entTipoTxn['txn_type'].map(len).max()

18

In [22]:
dtypes = [INTEGER,VARCHAR(18)]
entTipoTxn.to_sql(name='tbl_txn_type',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entTipoTxn.columns,dtypes)))

3

### Código Postal

In [184]:
entZip = data[['Merchant City','Merchant State','Zip']].drop_duplicates().dropna().reset_index(drop=True)
entZip['Zip'] = entZip['Zip'].map(float).astype(int).map(lambda x: f'{x:05d}')
entZip = entZip.merge(entTerritorio,how='inner',left_on=['Merchant State'],right_on=['territory_code'])
entZip = entZip.merge(aux[['id','territory_city','id_territory']],how='inner',left_on=['Merchant City','id'],right_on=['territory_city','id_territory'])
entZip = entZip[['Zip','id_y']]
entZip.columns = ['zip_code','id_city']
entZip.insert(0,'id',(entZip.index+1).astype(int))
entZip.head()

,id,zip_code,id_city
0,1,91750,1
1,2,91754,2
2,3,91755,2
3,4,91752,4
4,5,91765,5


In [185]:
dtypes = [INTEGER,CHAR(5),INTEGER]
entZip.to_sql(name='tbl_zip',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entZip.columns,dtypes)))

27321

### MCC - Merchant Category Code

In [55]:
mcc = data[['MCC']].drop_duplicates().dropna().reset_index(drop=True).astype(int)

In [73]:
entMCC = pd.read_csv('https://raw.githubusercontent.com/greggles/mcc-codes/main/mcc_codes.csv')
entMCC['mcc'] = entMCC['mcc'].map(int)
entMCC= entMCC[['mcc','edited_description']]
entMCC.columns = ['mcc','mcc_description']
entMCC.insert(0,'id',entMCC.index+1)
entMCC.head()

,id,mcc,mcc_description
0,1,742,Veterinary Services
1,2,763,Agricultural Co-operatives
2,3,780,"Horticultural Services, Landscaping Services"
3,4,1520,General Contractors-Residential and Commercial
4,5,1711,Air Conditioning Contractors – Sales and Insta...


In [76]:
entMCC['mcc_description'].map(len).max()

142

In [77]:
dtypes = [INTEGER,INTEGER,VARCHAR(150)]
entMCC.to_sql(name='tbl_mcc',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entMCC.columns,dtypes)))

981

### User

In [89]:
entUser = data[['User']].drop_duplicates().reset_index(drop=True)
entUser.columns =['id']

dtypes = [INTEGER]
entUser.to_sql(name='tbl_user',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entUser.columns,dtypes)))

2000

### Card

In [91]:
entCard = data[['User','Card']].drop_duplicates().reset_index(drop=True)
entCard.columns = ['id_user','card_number']
entCard.insert(0,'id',entCard.index+1)
entCard.head()

,id,id_user,card_number
0,1,0,0
1,2,0,1
2,3,0,2
3,4,0,3
4,5,0,4


In [92]:
dtypes = [INTEGER]*3
entCard.to_sql(name='tbl_card',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entCard.columns,dtypes)))

6139

### Errores 

In [103]:
entError = data[['id','Errors?']].dropna().reset_index(drop=True)
errors = [f'error_{i+1}' for i in range(3)]
entError[errors] = entError['Errors?'].map(lambda x:x.split(',')).apply(pd.Series)
entError.head()
entError = entError.melt(id_vars=['id'],value_vars=errors).dropna().reset_index(drop=True).drop('variable',axis=1)
entError = entError[['value']].drop_duplicates().reset_index(drop=True) 

,value
0,Technical Glitch
1,Insufficient Balance
2,Bad PIN
3,Bad Expiration
4,Bad Card Number
5,Bad CVV
6,Bad Zipcode


In [105]:
entError.columns = ['error_description']
entError.insert(0,'id',entError.index+1)
entError.head()

,id,error_description
0,1,Technical Glitch
1,2,Insufficient Balance
2,3,Bad PIN
3,4,Bad Expiration
4,5,Bad Card Number


In [106]:
dtypes = [INTEGER,VARCHAR(20)]
entError.to_sql(name='tbl_error',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entError.columns,dtypes)))

7

### Transacción 

In [108]:
data.head(2)

,id,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,1,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,2,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No


In [203]:
entTxn = data.copy()
entTxn['Amount'] = entTxn['Amount'].map(lambda x: x.replace('$','')).map(float)
entTxn['Zip'] =entTxn['Zip'].fillna(99999.0).astype(float).astype(int)
entTxn['Zip'] = np.where(entTxn['Zip']==99999,None,entTxn['Zip'].map(lambda x: f'{x:05d}'))
entTxn['MCC'] = entTxn['MCC'].fillna(9999).astype(int)
entTxn['MCC'] = np.where(entTxn['MCC']==9999,None,entTxn['MCC'])

entTxn['Year'] = entTxn['Year'].astype(str)
entTxn['Month'] = entTxn['Month'].astype(int).map(lambda x: f'{x:02d}')
entTxn['Day'] = entTxn['Month'].astype(int).map(lambda x: f'{x:02d}')

entTxn['txn_date'] = entTxn[['Year','Month','Day']].apply('-'.join,axis=1)
entTxn['ts'] = entTxn[['txn_date','Time']].apply(' '.join,axis=1)

entTxn['ts'] = pd.to_datetime(entTxn['ts'],format='%Y-%m-%d %H:%M')
entTxn.drop(['Year','Month','Day','Time','txn_date','Errors?'],axis=1,inplace=True)

entTxn.rename(columns={'User':'id_user','Card':'card_number'},inplace=True) 
entTxn = entTxn.merge(entCard.rename(columns={'id':'id_card'}),how='left',on=['id_user','card_number'])
entTxn.drop('card_number',axis=1,inplace=True)

entTxn.rename(columns={'Use Chip':'txn_type'},inplace=True) 
entTxn = entTxn.merge(entTipoTxn.rename(columns={'id':'id_txn_type'}),how='left',on=['txn_type'])
entTxn.drop('txn_type',axis=1,inplace=True)

entTxn.rename(columns={'Merchant Name':'merchant_name'},inplace=True) 
entTxn = entTxn.merge(entComercio.rename(columns={'id':'id_comercio'}),how='left',on=['merchant_name'])
entTxn.drop('merchant_name',axis=1,inplace=True)

entTxn = entTxn.merge(entTerritorio.rename(columns={'id':'id_territory'}),
                      how='left',
                      left_on=['Merchant State'],right_on=['territory_code'],indicator=True)

entTxn['Merchant State'] = np.where(entTxn['_merge']=='left_only',entTxn['Merchant State'],None)
entTxn = entTxn.merge(entTerritorio.rename(columns={'id':'id_territory'}),
                      how='left',
                      left_on=['Merchant State'],right_on=['territory_name'])
entTxn['id_territory'] = (np.where(entTxn['id_territory_x'].isnull(),entTxn['id_territory_y'],entTxn['id_territory_x'])).astype(int)
entTxn.drop(['id_territory_x','territory_code_x','territory_name_x','_merge','id_territory_y','territory_code_y','territory_name_y'],axis=1,inplace=True)
entTxn.drop(['Merchant City','Merchant State'],axis= 1,inplace=True)   

entTxn.rename(columns={'Zip':'zip_code'},inplace=True) 
entTxn = entTxn.merge(entZip.rename(columns={'id':'id_zip'}),
                      how='left',
                      on='zip_code')
entTxn.drop('zip_code',axis=1,inplace=True)

entTxn.rename(columns={'MCC':'mcc'},inplace=True) 
entTxn = entTxn.merge(entMCC.rename(columns={'id':'id_mcc'}).drop('mcc_description',axis=1),
                      how='left',
                      on='mcc')
entTxn.drop('mcc',axis=1,inplace=True)

entTxn.rename(columns={'Is Fraud?' : 'is_fraud'},inplace=True)
entTxn['is_fraud'] = entTxn['is_fraud']=='Yes'

entTxn.rename(columns={'Amount':'amount'},inplace=True)

In [204]:
dtypes = [INTEGER]*2+[FLOAT,BOOLEAN,DATETIME]+[INTEGER]*7
entTxn.to_sql(name='tbl_txn',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entTxn.columns,dtypes)))

### Transacción-Error

In [195]:
entErrTxn = data[['id','Errors?']].dropna().reset_index(drop=True)
errors = [f'error_{i+1}' for i in range(3)]
entErrTxn[errors] = entErrTxn['Errors?'].map(lambda x:x.split(',')).apply(pd.Series)
entErrTxn = entErrTxn.melt(id_vars=['id'],value_vars=errors).dropna().reset_index(drop=True).drop('variable',axis=1)
entErrTxn = entErrTxn.merge(entError.rename(columns={'id':'id_error'}),left_on='value',right_on='error_description').drop(['value','error_description'],axis=1).rename(columns={'id':'id_txn'})

In [202]:
dtypes = [INTEGER]*2
entErrTxn.to_sql(name='tbl_txn_error',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entErrTxn.columns,dtypes)))

390151